In [1]:
import pandas as pd
import numpy as np

In [2]:
df  = pd.read_csv('IMDB Dataset.csv')
df =df.iloc[:10000]
df.shape

(10000, 2)

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.drop_duplicates(inplace=True)
df.shape

(9983, 2)

In [5]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'),'',raw_text)
    return cleaned_text


In [6]:
df['review'] = df['review'].apply(remove_tags)

In [7]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [8]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [9]:
from nltk.corpus import stopwords
swe = stopwords.words("english")

In [10]:
df['review']=df['review'].apply(lambda x:[item for item in x.split() if item not in swe]).apply(lambda x : " ".join(x))

In [11]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import gensim

In [12]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [13]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [14]:
model.build_vocab(story)

In [15]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(5875273, 6212140)

In [16]:
len(model.wv.index_to_key)

31845

In [17]:
def document_vector(doc):
    doc =[word for word in doc.split() if word in model.wv.index_to_key ]
    return np.mean(model.wv[doc],axis=0)

In [18]:
document_vector(df['review'].values[0])

array([-0.17718686,  0.4664186 ,  0.14830631,  0.2817664 , -0.1566359 ,
       -0.6298632 ,  0.23839004,  0.9460216 , -0.3686266 , -0.22752471,
       -0.34028175, -0.44364977,  0.11369275,  0.0908562 ,  0.21275018,
       -0.11724677, -0.00558348, -0.35362902, -0.10681229, -0.6448627 ,
        0.01520701,  0.29124337,  0.13166143, -0.23255655, -0.2812769 ,
        0.00473497, -0.30008528, -0.0389306 , -0.3568865 ,  0.05536943,
        0.32624534, -0.03233646,  0.17604963, -0.280831  , -0.13998955,
        0.43494743,  0.07901493, -0.38113058, -0.25597483, -0.77152234,
        0.13010207, -0.23151997,  0.06465267, -0.06035282,  0.50842476,
       -0.19848987, -0.2664495 , -0.03970525,  0.10067033,  0.35941994,
        0.08937433, -0.31646833, -0.41516823, -0.07425223, -0.11835513,
        0.20304376,  0.20872101,  0.03342823, -0.24468195,  0.0980118 ,
        0.0562724 ,  0.10129029,  0.00625083, -0.07019447, -0.42890477,
        0.21618578,  0.0416024 ,  0.07298215, -0.36254585,  0.27

In [19]:
from tqdm import  tqdm

In [20]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [14:09<00:00, 11.75it/s]


In [21]:
X = np.array(X)
X.shape

(9983, 100)

In [23]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y  = encoder.fit_transform(df['sentiment'])

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7786680020030045